In [ ]:
import sympy as sym
import numpy as np
from scipy import linalg

In [ ]:
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 1.1

# roll, pitch, yaw angles
phi, theta, psi = sym.symbols('phi, theta, psi')

# angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# resultant torques
lt = sym.nsimplify(lw) * sym.sqrt(2) / 2
T1 = - tau_1 * sym.Matrix([[lt], [0], [lt]])
T2 = - tau_2 * sym.Matrix([[-lt], [0], [lt]])
T3 = - tau_3 * sym.Matrix([[0], [lt], [lt]])
T4 = - tau_4 * sym.Matrix([[0], [-lt], [lt]])
T = T1 + T2 + T3 + T4

# parameters
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# rotation matrices
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry * Rz).T * ex, Rz.T * ey, ez).inv(), full=True)

# euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

In [ ]:
f

In [ ]:
# Make f an executable function
f_num = sym.lambdify((phi, theta, psi, w_x, w_y, w_z,tau_1, tau_2, tau_3, tau_4),f)

In [ ]:
#Equilibrium values

phi_e = 0
theta_e = 0
psi_e = 0
w_x_e = 0
w_y_e = 0
w_z_e = 0
tau_1_e = 0
tau_2_e = 0
tau_3_e = 0
tau_4_e = 0

In [ ]:
feq = f_num(phi_e, theta_e
, psi_e
, w_x_e
, w_y_e
, w_z_e
, tau_1_e
, tau_2_e
, tau_3_e
, tau_4_e)

In [ ]:
feq #Works perfectly

In [ ]:
# Now we linearize our system about the equilibrium point
# Linearizing the system

#Now we find the Jacobians
f_jacob_x = f.jacobian([phi, theta, psi, w_x, w_y, w_z])
f_jacob_u = f.jacobian([tau_1, tau_2, tau_3, tau_4])

#And then we find functions for A and B using this jacobian
A_num = sym.lambdify((phi, theta, psi, w_x, w_y, w_z,tau_1, tau_2, tau_3, tau_4),f_jacob_x)
B_num = sym.lambdify((phi, theta, psi, w_x, w_y, w_z,tau_1, tau_2, tau_3, tau_4),f_jacob_u)

#Finally, we find the linearized state space model by evaluating A_num and B_num at the equilibrium points
A = A_num(phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e,tau_1_e, tau_2_e, tau_3_e, tau_4_e).astype(float)
B = B_num(phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e,tau_1_e, tau_2_e, tau_3_e, tau_4_e).astype(float)

In [ ]:
A.tolist()

In [ ]:
B.tolist()

In [ ]:
## Checking for controllability
# Find the number of states
n = A.shape[0]

# Initialize W with its first column
W = B

# Create W one column at a time by iterating over i from 1 to n-1
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    W = np.block([W, col])

In [ ]:
# What about the rank?
print(np.linalg.matrix_rank(W))
print(W.shape)
#The rank is six, so the system is controllable!

In [ ]:
Qc = np.diag([0.005,1.,1.,0.075,0.4,0.4]) #Roll, pitch, yaw, omegax, omegay, omegaz
Rc = np.diag([1.,1.,1.,1.]) #Tau 1 to tau 4
# We can also define an LQR function in the following way:
def lqr(A,B,Q,R):
    P = linalg.solve_continuous_are(A,B,Q,R)
    K = linalg.inv(R) @ B.T @ P
    return K
# The function above can be used for both controller and observer design
K = lqr(A,B,Qc,Rc)
K.tolist()

In [ ]:
# We can directly start designing an LQR controller for the system. It is independent of the observer
s = np.array([1.0j, -1.0j,1.0j, -1.0j,1.0j, -1.0j])
while (s.imag != 0).all(): 
    angweight = np.random.rand()
    velweight = np.random.rand()
    Qc = 10*np.diag([angweight,angweight,angweight,velweight,velweight,velweight]) #Roll, pitch, yaw, omegax, omegay, omegaz
    Rc = np.diag([1.,1.,1.,1.]) #Tau 1 to tau 4
    # We can also define an LQR function in the following way:
    def lqr(A,B,Q,R):
        P = linalg.solve_continuous_are(A,B,Q,R)
        K = linalg.inv(R) @ B.T @ P
        return K
    # The function above can be used for both controller and observer design
    K = lqr(A,B,Qc,Rc)
    K.tolist()
    F = A - B@K
    s = linalg.eigvals(F)
K.tolist()
s

In [ ]:
# Now we work on the Observer
alpha, delta = sym.symbols('alpha, delta')
# Scope radius
r = 0.8 / 2.1
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rx * Ry * Rz

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                       [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)

#This is the output equation: y = Cx, where x is the state vector and y is the position of the star given in y-pos and z-pos vars per star
#The following line is taken from the Day30 in-class code

In [ ]:
g

In [ ]:
#Make G a combination of 3 stars with set position data
alpha1 = 0.
alpha2 = 0.15
alpha3 = 0.
alpha4 = -0.2
alpha5 = 0.
delta1 = 0.
delta2 = 0.
delta3 = 0.15
delta4 = 0.
delta5 = -0.2

G = np.block([[g.subs([(alpha, alpha1), (delta, delta1)])],[g.subs([(alpha, alpha2), (delta, delta2)])],[g.subs([(alpha, alpha3), (delta, delta3)])],[g.subs([(alpha, alpha4), (delta, delta4)])],[g.subs([(alpha, alpha5), (delta, delta5)])]])
G = sym.Matrix(G)
G

In [ ]:
# We also need to find the equilibrium value of this system and subtract that from z to get y
G_num = sym.lambdify((phi, theta, psi),G)
Geq = G_num(phi_e, theta_e, psi_e)
Geq.tolist()

In [ ]:
#We now create a numerical function for C, that will then be evaluated at the equilibrium point
C_num = sym.lambdify((phi, theta, psi), G.jacobian([phi, theta, psi, w_x, w_y, w_z]))
C = C_num(phi_e, theta_e, psi_e)
C.tolist()

In [ ]:
# We can check for the observability of this system
Wo = C
for i in range(1,n):
    row = C @ np.linalg.matrix_power(A,i)
    Wo = np.block([[Wo],[row]])
rank = np.linalg.matrix_rank(Wo)
rank
# The rank of the system is the same as the number of states, so the system is observable.


In [ ]:
# Designing an optimal controller
Qo = np.diag(np.ones(2*5)) # (2n)x(2n) for an n star system
Ro = np.diag([1.,1.,1.,1.,1.,1.])

L = lqr(A.T, C.T, linalg.inv(Ro), linalg.inv(Qo)).T
L.tolist()

In [ ]:
p = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
z = np.array([p]).T
z